In [1]:
import spacy
import pandas as pd
import docx

from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from google.cloud import translate_v2
from IPython.display import HTML as html_print

In [2]:
#instantiate nlp
nlp = spacy.load('en_core_web_sm')

In [3]:
#实验
test = 'Grand Slams, John Roland'

test_doc = nlp(test)

for token in test_doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

Grand PROPN NNP compound Xxxxx True False
Slams PROPN NNP ROOT Xxxxx True False
, PUNCT , punct , False False
John PROPN NNP compound Xxxx True False
Roland PROPN NNP appos Xxxxx True False


In [4]:
spacy.explain('appos')

'appositional modifier'

In [6]:
def read_docx(filename):
    doc = docx.Document(filename)
    
    sentences = []
    
    for para in doc.paragraphs:
        if para.text != '':
            sentences.append(para.text.replace('\xa0', ''))
    
    return sentences

In [7]:
sent_list = read_docx('scripts/script02.docx')

In [8]:
#build patterns
match_pattern = [{'LEMMA': 'match', 'POS': {'IN': ['NOUN']}}]
agressive_pattern = [{'LEMMA': 'aggressive'}]
proper_name_pattern = [{'IS_TITLE': True, 'POS': 'PROPN'}, {'IS_TITLE': True}] #需要跟其它两个大写词组合区分
fedcup_pattern = [{'LOWER': 'fed'}, {'LOWER': 'cup'}]
grandslam_pattern = [{'LOWER': 'grand'}, {'LOWER': {'IN': ['slam', 'slams']}}]
rally_pattern = [{'LEMMA': 'rally'}]
doublefault_pattern = [{'LOWER': 'double'}, {'LOWER': 'faults'}]

In [9]:
#instantiate a matcher
matcher = Matcher(nlp.vocab, validate=True)

In [10]:
#add the matche patterns
matcher.add('比赛', [match_pattern])
matcher.add('打得很凶', [agressive_pattern])
matcher.add('球员名字', [proper_name_pattern])
matcher.add('联合会杯', [fedcup_pattern])
matcher.add('大满贯比赛', [grandslam_pattern])
matcher.add('回合', [rally_pattern])
matcher.add('双误', [doublefault_pattern])

In [30]:
#有match的
#code credit: https://stackoverflow.com/questions/51037383/how-do-i-add-matches-as-entities-and-visualize-it-in-spacy
for sent in sent_list:
    doc = nlp(sent)
    if matcher(doc):
        span = [Span(doc, start, end, label=match_id) for match_id, start, end in matcher(doc)]
        doc.ents = span
    #         doc.ents = list(doc.ents) + [span]
        #print([(ent.text, ent.label_) for ent in doc.ents])
        displacy.render(doc, style='ent')

ValueError: [E1010] Unable to set entity information for token 39 which is included in more than one span in entities, blocked, missing or outside.

In [13]:
#实验二
test = """
So I was a little bit trying to get the rhythm to play a little bit more aggressive, 
maybe a little bit longer rallies, that I can go to the game and be able to play.
"""
test_doc = nlp(test)
matches = matcher(test_doc)

In [27]:
span = [Span(test_doc, start, end, label=match_id) for match_id, start, end in matches]

In [28]:
test_doc.ents = span

In [29]:
displacy.render(test_doc, style='ent')

In [ ]:
from spacy.tokens.span_group import SpanGroup

In [ ]:
sg = SpanGroup(all_doc)

In [ ]:
type(sg)

In [ ]:
sg.

In [ ]:
for sent in sent_list:
    doc = nlp(sent)
    if matcher(doc):
        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=match_id)
            doc.ents = [span]
        displacy.render(doc, style='ent')

In [ ]:
all_para = ' '.join(read_docx('scripts/script02.docx'))

In [ ]:
all_doc = nlp(all_para)

In [ ]:
for match in matcher(all_doc):
    span = Span(doc, start, end, label=match_id)
    sg.append(span)
#     for match_id, start, end in match:
#         span = Span(doc, start, end, label=match_id)
#         print(span)

# all_match = matcher(all_doc)

In [ ]:
#没有match
for sent in sent_list:
    if not matcher(nlp(sent)):
        print(sent)